# Import and Install Dependencies

In [1]:
# !pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

In [13]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt

import time
import mediapipe as mp

# Landmark Set up

In [27]:
#Creation of the Holistic Model
mp_holistic = mp.solutions.holistic
#Allows for Drawing Capabilities
mp_drawing = mp.solutions.drawing_utils

In [28]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [29]:
def draw_landmarks(image, results):
    #code to draw landmarks onto the images then use it to process the videos visually
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Draw face contours
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [30]:
#Creation of styled landmarks 
def draw_styled_landmarks(image, results):
    # Draw face mesh connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [31]:
cap = cv2.VideoCapture(0)

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Extract Landmark Values

In [32]:
#Extraction of keypoints for a single frame
#extract out key points from pose, frame, lh and rh

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Folder set up for simple Collection

In [79]:
os.makedirs('MP_Data')

In [80]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Start', 'Others'])

# fifty videos worth of data
no_sequences = 50

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 50

In [81]:
#Create a file with a total of 50 videos, 30 frames in length, 2 

for action in actions: 
    for sequence in range(no_sequences):
        try: 
            #make the directories
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Data Collection Procedure

In [82]:
#take each action, each video adn each frame to create a folder with the respective frames
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions (Actions being the thumbs up and the head nod, and any other action)
    # In this case, we only have 2 actions
    for action in actions:
        # Loop through sequences aka videos (There will be a total of 50 videos)
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    # show starting collection for the start of the video
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    #Input the video being collected
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(300)
                else: 
                    # After the first frame, it will simply just be the video
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                # Save the numpy arrays into a specific path
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyArf8ollWindows()

# Creation of Labels

In [83]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [84]:
label_map = {label:num for num, label in enumerate(actions)}

In [85]:
sequences, labels = [], []
#For all the actions >> in this case there is only 2
#the first is the gesture we want to detect, the second is other gestures
for action in actions:
    # list of files inside each of the action libraries and files
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        #for each frame within the video
        for frame_num in range(sequence_length):
            #loading the entire path
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            #Append into the window for the entire video
            window.append(res)
        #Add the video into sequences
        sequences.append(window)
        #Add the labels for the video to prepare the data for training
        labels.append(label_map[action])

In [86]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)


In [104]:
X_train[1]

array([[ 0.61138654,  0.17732976, -0.40876406, ...,  0.51870573,
         0.60247809,  0.00175538],
       [ 0.6104669 ,  0.1772774 , -0.40932861, ...,  0.50853109,
         0.61154467, -0.00194538],
       [ 0.60999334,  0.17718831, -0.4100554 , ...,  0.50934422,
         0.61000538, -0.00245177],
       ...,
       [ 0.60945499,  0.17700574, -0.38661927, ...,  0.51030588,
         0.62263286, -0.00560592],
       [ 0.60944474,  0.17700352, -0.39412916, ...,  0.50907487,
         0.62106997, -0.00226652],
       [ 0.60927713,  0.17692433, -0.39857551, ...,  0.50842142,
         0.61739284, -0.00392845]])

In [105]:
X_train[0]


array([[ 0.55486023,  0.18433163, -0.32251036, ...,  0.44351724,
         0.33278453, -0.01529845],
       [ 0.55429518,  0.1842262 , -0.32203376, ...,  0.4430148 ,
         0.3338322 , -0.01378378],
       [ 0.554084  ,  0.18418197, -0.31923005, ...,  0.44117719,
         0.33337113, -0.01457913],
       ...,
       [ 0.5533663 ,  0.18114309, -0.34027532, ...,  0.43217036,
         0.37379843, -0.01575904],
       [ 0.55336297,  0.18123321, -0.34218633, ...,  0.43187788,
         0.3741014 , -0.01636777],
       [ 0.55333161,  0.18127362, -0.34969094, ...,  0.43136731,
         0.37623882, -0.01616663]])

In [101]:
X_train[0]

array([[ 0.55486023,  0.18433163, -0.32251036, ...,  0.44351724,
         0.33278453, -0.01529845],
       [ 0.55429518,  0.1842262 , -0.32203376, ...,  0.4430148 ,
         0.3338322 , -0.01378378],
       [ 0.554084  ,  0.18418197, -0.31923005, ...,  0.44117719,
         0.33337113, -0.01457913],
       ...,
       [ 0.5533663 ,  0.18114309, -0.34027532, ...,  0.43217036,
         0.37379843, -0.01575904],
       [ 0.55336297,  0.18123321, -0.34218633, ...,  0.43187788,
         0.3741014 , -0.01636777],
       [ 0.55333161,  0.18127362, -0.34969094, ...,  0.43136731,
         0.37623882, -0.01616663]])

# Build LSTM Framework

In [90]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [91]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [92]:
actions.shape[0]

2

In [93]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [94]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [95]:
y_test.shape

(10, 2)

In [97]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - categorical_accuracy: 0.5072 - loss: 0.6909
Epoch 2/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - categorical_accuracy: 0.5697 - loss: 0.6879
Epoch 3/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - categorical_accuracy: 0.5150 - loss: 0.6890
Epoch 4/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - categorical_accuracy: 0.5272 - loss: 0.6889
Epoch 5/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - categorical_accuracy: 0.6304 - loss: 0.6843
Epoch 6/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - categorical_accuracy: 0.6041 - loss: 0.6816
Epoch 7/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - categorical_accuracy: 0.6987 - loss: 0.6798
Epoch 8/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - categorical_accuracy: 0.8213 - loss: 0.6714
Epoch 9/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - categorical_accuracy: 0.7602 - loss: 0.6589
Epoch 10/2000
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - categorical_accuracy: 0.5045 - loss: 0.7050
Epoch 11/2000
3/3 ━━━━━━━━━━━

In [98]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [99]:
res

array([[0.4858748 , 0.51412517],
       [0.4858748 , 0.51412517],
       [0.4858748 , 0.51412517],
       [0.4858748 , 0.51412517],
       [0.4858748 , 0.51412517],
       [0.4858748 , 0.51412517],
       [0.4858748 , 0.51412517],
       [0.4858748 , 0.51412517],
       [0.4858748 , 0.51412517],
       [0.4858748 , 0.51412517]], dtype=float32)

In [53]:
y_test

array([[1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1]])

In [9]:
# model.save('action.h5')
model.load_weights('action.h5')

NameError: name 'model' is not defined

In [106]:
from scipy import stats

In [107]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [108]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti